In [1]:
import json
import matplotlib.pyplot as plt

COLORS = [
    'red', 'blue', 'green', 'purple', 'orange', 'darkred',
    'darkblue', 'darkgreen', 'cadetblue',
    'white', 'pink',
    'gray', 'black'
]

def plot_markers(objects, marker=".", size=20, color="blue"):
    latitudes = [x["Latitude"] for x in objects]
    longitudes = [x["Longitude"] for x in objects]
    plt.scatter(latitudes, longitudes, marker=marker, s=size, color=color)
    

def plot_route(route, size=20, color="green"):
    points = route["Points"]
    latitudes = [x["Latitude"] for x in points]
    longitudes = [x["Longitude"] for x in points]
    plt.plot(latitudes, longitudes, color=color)
    for i in range(len(latitudes) - 1):
        x = (latitudes[i] + latitudes[i+1]) / 2
        y = (longitudes[i] + longitudes[i+1]) / 2
        direction_x = latitudes[i + 1] - latitudes[i]
        direction_y = longitudes[i + 1] - longitudes[i]
        length = (direction_x ** 2 + direction_y ** 2) ** 0.5
        if length == 0:
            continue
        direction_x /= length
        direction_y /= length
        plt.arrow(
            x, y, 
            direction_x, direction_y,
            shape='full', lw=0, 
            length_includes_head=True, head_width=size,
            color=color
        )
        
def plot_solution(input_file, figsize=(12, 7), title="Solution", output=None, show=True):
    with open(input_file) as f:
        solution = json.load(f)

    fig = plt.figure(figsize=(figsize))
    plt.suptitle(f"Total distance: {solution['TotalDistance']}")
    plt.title(title)
    plot_markers(solution["Containers"], marker="X", size=100, color="blue")
    plot_markers(solution["Trucks"], marker="s", size=100, color="red")
    plot_markers(solution["Landfills"], marker="h", size=100, color="green")
    for i, route in enumerate(solution["Routes"]):
        plot_route(route, size=10, color=COLORS[i % len(COLORS)])
    if show:
        plt.show()
    if output is not None:
        plt.savefig(output)
    fig.clear()
    plt.close(fig)

In [2]:
import subprocess
import sys
import os
import time


PROGRAM_PATH = "./routing-problem"


def create_dir(path):
    if os.path.exists(path):
        return

    os.makedirs(path)


def run(params):
    args = [f"--{name}={value}" for name, value in params.items()]
    command = f"{PROGRAM_PATH} {' '.join(args)}"
    res = os.system(command)
    if res != 0:
        print("ERROR")
        print("Command:", command)
        print("RES:", res)
        
def generate_params(search_data):
    keys = []
    all_params = [] 
    for key in search_data:
        if type(search_data[key]) == list:
            all_params.append(search_data[key])
        else:
            all_params.append([search_data[key]])
        keys.append(key)
    finished = False
    
    indices = [0] * len(keys)
    while not finished:
        params = {keys[i]: all_params[i][indices[i]] for i in range(len(keys))}
        yield params
        for i in range(len(keys)):
            index = len(keys) - i - 1
            if len(all_params[index]) - 1 == indices[index]:
                indices[index] = 0
                if index == 0:
                    finished = True
                    break
            else:
                indices[index] += 1
                break
                    
        
def perform_becnhmarks(searches_data, base_folder, benchmarks="benchmarks"):
    create_dir(base_folder)
    i = 0
    for search_data in searches_data:
        for params in generate_params(search_data):
            for file in os.listdir(benchmarks):
                for key, value in params.items():
                    if key == "output" or key == "input":
                        continue
                    print(f"{key}={value},", end=" ")
                    
                print("\nFile", file)
                params["output"] = f"{base_folder}/output_{i}.json"
                params["input"] = f"{benchmarks}/{file}"
                run(params)
                with open(params["output"]) as f:
                    output = json.load(f)
                for param in params:
                    output[param] = params[param]
                json_string = json.dumps(output, indent=4)
                with open(params["output"], "w") as f:
                    f.write(json_string)
                i += 1
            

In [ ]:
heuristics = [
    "SWAP_IN_EXPORTATION",
    "MOVE_IN_EXPORTATION",
    "REMOVE_LOOPS_SWAP_ANY_EXPORTATION",
    "REMOVE_LOOPS_MOVE_EXPORTATION",
    "REMOVE_LOOPS_SWAP_EXPORTATION",
    "ANY_EXPORTATION_PROB1",
    "ANY_EXPORTATION_PROB2",
    "ANY_EXPORTATION_FIXED1",
    "ANY_EXPORTATION_FIXED2",
    "ANY_MOVE_EXPORTATION_FIXED2",
    "ANY_EXPORTATION_TIME1",
    "ANY_EXPORTATION_TIME2",
    "ANY_EXPORTATION",
    "ANY_MOVE_EXPORTATION",
    "ANY_MOVE_SWAP",
]

searches = [
    {
        "search": "TABU",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER", "SIMPLE_GREEDY"],
        "tabu_queue_coef": [1.0, 3.0, 5.0],
        "iterations": [100, 500, 1000]
    },
    {
        "search": "ANNEALING",
        "heuristic": heuristics,
        "base_solution": ["CLUSTER", "SIMPLE_GREEDY"],
        "iterations": [50000, 100000],
        "max_negative_change": 0.2,
        "max_no_change_iterations": 100000
    },
]


perform_becnhmarks(searches, "results")

search=TABU, heuristic=SWAP_IN_EXPORTATION, base_solution=CLUSTER, tabu_queue_coef=1.0, iterations=100, 
File c20_t1_r1.json
5638.27
5221.23
search=TABU, heuristic=SWAP_IN_EXPORTATION, base_solution=CLUSTER, tabu_queue_coef=1.0, iterations=100, 
File c300_t6_r5.json
23927
23601.9
search=TABU, heuristic=SWAP_IN_EXPORTATION, base_solution=CLUSTER, tabu_queue_coef=1.0, iterations=100, 
File c100_t4_r1.json
15275.1
14490.8
search=TABU, heuristic=SWAP_IN_EXPORTATION, base_solution=CLUSTER, tabu_queue_coef=1.0, iterations=100, 
File c5_t1_r1.json
3443.69
3218.05
search=TABU, heuristic=SWAP_IN_EXPORTATION, base_solution=CLUSTER, tabu_queue_coef=1.0, iterations=100, 
File c100_t3_r3.json
10568.8
10322.3
search=TABU, heuristic=SWAP_IN_EXPORTATION, base_solution=CLUSTER, tabu_queue_coef=1.0, iterations=100, 
File c50_t2_r2.json
8330.52
7783.02
search=TABU, heuristic=SWAP_IN_EXPORTATION, base_solution=CLUSTER, tabu_queue_coef=1.0, iterations=100, 
File c300_t6_r3.json
24792.2
24196.2
search=TABU,

In [ ]:
import pandas as pd

def get_output_files(results_directory):
    for search in os.listdir(results_directory):
        if search == ".DS_Store":
            continue
        for directory in os.listdir(f"{results_directory}/{search}"):
            if directory == ".DS_Store":
                continue
            for output in os.listdir(f"{results_directory}/{search}/{directory}/output"):
                yield f"{results_directory}/{search}/{directory}/output/{output}"

dataframe_data = {
    "BaseSolution": [],
    "TotalDistance": [],
    "UnusedTrucksCount": [],
    "Search": [],
    "Heuristic": [],
    "OrphansCount": [],
}
names = []
iterations = []

for output_file in get_output_files("results"):
    with open(output_file) as f:
        solution_info = json.load(f)
    for key in dataframe_data:
        dataframe_data[key].append(solution_info[key])
    iteration = int(output_file.split("/")[2].split("-")[0])
    iterations.append(iteration)
    names.append(output_file.split("/")[-1].split(".")[0])
    
dataframe_data["Name"] = names
dataframe_data["Iterations"] = iterations

In [ ]:
df = pd.DataFrame(data=dataframe_data)
df.to_csv("all_results.csv")
df.head()

In [ ]:
for name in set(names):
    file_data = df[df["Name"] == name]
    row = file_data.nsmallest(1, "TotalDistance")
    print(f"{name}: {row['Search'].values[0]}, {row['BaseSolution'].values[0]}, {row['Heuristic'].values[0]}")

In [ ]:
df[df["Search"] == "ANNEALING"]

In [ ]:
df[df["Search"] == "ANNEALING"]

In [ ]:
for name in sorted(list(set(names))):
    file_data = df[df["Name"] == name]
    file_data = file_data[file_data["Search"] == "NO_SEARCH"]
    file_data = file_data[file_data["BaseSolution"] == "CLUSTER"]
    row = file_data.nsmallest(1, "TotalDistance")
    print(f"{name}: {row['Search'].values[0]},"
          f"{row['BaseSolution'].values[0]},"
          f" {row['Heuristic'].values[0]}, Dist: {row['TotalDistance'].values[0]}")

In [ ]:
for name in sorted(list(set(names))):
    file_data = df[df["Name"] == name]
    file_data = file_data[file_data["Search"] == "ANNEALING"]
    row = file_data.nsmallest(1, "TotalDistance")
    print(f"{name}: {row['Search'].values[0]},"
          f"{row['BaseSolution'].values[0]},"
          f" {row['Heuristic'].values[0]}, Dist: {row['TotalDistance'].values[0]} {row['Iterations'].values[0]}")

In [ ]:
for name in sorted(list(set(names))):
    file_data = df[df["Name"] == name]
    file_data = file_data[file_data["Search"] == "TABU"]
    row = file_data.nsmallest(1, "TotalDistance")
    print(f"{name}: {row['Search'].values[0]},"
          f"{row['BaseSolution'].values[0]},"
          f" {row['Heuristic'].values[0]}, Dist: {row['TotalDistance'].values[0]} {row['Iterations'].values[0]}")

In [ ]:
plot_solution(
        f"output.json",
        figsize=(15, 10),
        title="",
        output=f"test.png",
        show=True
        )

In [ ]:
plot_solution(
        f"output.json",
        figsize=(15, 10),
        title="",
        output=f"test.png",
        show=True
        )

In [ ]:
with open("results/TABU/500-CLUSTER-2-MOVE_IN_EXPORTATION/output/c300_t6_r5.json.json") as f:
    data = json.load(f)
data

In [ ]:
def calc_dist(filename):
    with open(filename) as f:
        data = json.load(f)

    dist = 0
    i = 0
    for route in data["Routes"]:
        prev_point = None
        for point in route["Points"]:
            i += 1
            if prev_point is None:
                prev_point = point
                continue

            dist += ((point["Latitude"] - prev_point["Latitude"]) ** 2 
                     + (point["Longitude"] - prev_point["Longitude"]) ** 2) ** 0.5
            prev_point = point
    return dist, data["TotalDistance"]


calc_dist("results/TABU/500-SIMPLE_GREEDY-1-MOVE_IN_EXPORTATION/output/c50_t2_r1.json.json")

In [ ]:
calc_dist("../project/output.json")

In [ ]:
plot_solution(
        f"results/output_1.json",
        figsize=(15, 10),
        title="",
        output=f"test.png",
        show=True
        )

In [ ]:
plot_solution(
        f"results/output_0.json",
        figsize=(15, 10),
        title="",
        output=f"test.png",
        show=True
        )